In [1]:
import pandas as pd
import numpy as np
import keras
import gc

Using TensorFlow backend.


In [2]:
work_path = '/home/mmde-lab/s3/59e875d818c7b/'
deal_path = 'kddi_r_deal.tsv_20180328.gz'
deal_profile_path = 'kddi_r_deal_profile_20180427.tsv.gz'
deal_profile_path2 = 'kddi_r_deal_profile_20171222.tsv.gz'
df_deal = pd.read_csv(work_path + deal_path, sep = '\t', compression='gzip')
df_dealprofile = pd.read_csv(work_path + deal_profile_path, sep = '\t', compression='gzip')
df_dealprofile2 = pd.read_csv(work_path + deal_profile_path2, sep = '\t', compression='gzip')

df_deal = df_deal.dropna(axis=1, thresh= 270000)

df_dealprofile = df_dealprofile.append(df_dealprofile2)
df_dealprofile = df_dealprofile[['deal_id', 'limitation_cd', 'fixed_about_reserve_add_flg','reservation_visible_flg', 'reservation_required_flg','fixed_about_gift_add_flg','del_flg']]

dealpro_cat_col = ['limitation_cd', 'fixed_about_reserve_add_flg','reservation_visible_flg', 'reservation_required_flg','fixed_about_gift_add_flg','del_flg']
deal_num_col = ['deal_price','max_amount', 'min_amount', 'set_amount', 'remind_mail', 'after_purchase_remind_mail','max_amount_per_account', 'min_amount_per_account',]
deal_cat_col = ['branch_deal_flg', 'deal_type_cd', 'sales_method', 'deal_category_cd', 'deal_status', 'fixed_price_name', 'usable_day_visible_flg', 'ordered_flg', 'soldout_flg','strategy_flg', 'account_duplicate_chk_flg', 'template_flg', 'mobile_usable_flg', 'booking_system_flg', 'np_cart_flg', 'sales_progress_flg', 'fixed_score_boost_flg', 'wm_fixed_score_boost_flg', 'featured_flg', 'preferential_flg', 'loto_flg', 'deal_visible_cd', 'sku_default_visible_flg', 'purchase_count_visible_flg', 'api_visible_flg', 'correspondence_education_flg', 'want_visible_flg', 'dummy_deal_flg', 'max_amount_upd_flg', 'furusato_flg']

df_deal = df_deal[['deal_id'] + deal_num_col + deal_cat_col]

from gensim import corpora
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.utils import to_categorical

def cate2OneHot(Cfeature):
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(Cfeature)
    encoded = to_categorical(integer_encoded)
    return pd.DataFrame(encoded)

df_dealprofile = df_dealprofile.dropna()
df_deal = df_deal.dropna()

df_dealprofile = df_dealprofile.reset_index(drop=True)
df_deal = df_deal.reset_index(drop=True)

for column in dealpro_cat_col:
    df_dealprofile = pd.concat([df_dealprofile,cate2OneHot(df_dealprofile[column]).rename(columns = lambda x: str(column)+'_' + str(x))], axis = 1)

for column in deal_cat_col:
    df_deal = pd.concat([df_deal,cate2OneHot(df_deal[column]).rename(columns = lambda x: str(column)+'_' + str(x))], axis = 1)

df_dealprofile.drop(dealpro_cat_col, axis=1,inplace=True)
df_deal.drop(deal_cat_col, axis=1, inplace=True)

from sklearn.preprocessing import StandardScaler, MinMaxScaler
data_scaler = StandardScaler()

df_deal[deal_num_col] = data_scaler.fit_transform(df_deal[deal_num_col])

/home/mmde-lab/anaconda3/envs/whxPyEnv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (9,14,15,51,56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/mmde-lab/anaconda3/envs/whxPyEnv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (10,13,26,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/mmde-lab/anaconda3/envs/whxPyEnv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,3,9,10,13,17,18,24,25,26,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
def read_embedded_vector(cb_or_sb):
    if cb_or_sb == 'cb':
        df_deal_vec = pd.read_csv('df_item_cbwin5_trainset.csv.gz', compression='gzip')
        df_user_vec = pd.read_csv('df_user_cbwin5_trainset.csv.gz', compression='gzip')
        df_deal_vec.dealvec = df_deal_vec.dealvec.map(lambda x: x[2:-2].split())
    if cb_or_sb == 'sb':
        df_deal_vec = pd.read_csv('df_item_sbwin5_trainset.csv.gz', compression='gzip')
        df_user_vec = pd.read_csv('df_user_sbwin5_trainset.csv.gz', compression='gzip')
        df_deal_vec.dealvec = df_deal_vec.dealvec.map(lambda x: x[1:-1].split())
    df_deal_vec = df_deal_vec[['deal_id', 'dealvec']]
    df_user_vec = df_user_vec[['account_id', 'uservec']]
    df_user_vec.uservec = df_user_vec.uservec.map(lambda x: x[2:-2].split())

    for i in range(200):
        df_deal_vec['dealvec_' + cb_or_sb + str(i)] = df_deal_vec.dealvec.map(lambda x: float(x[i]))
    for i in range(200):
        df_user_vec['uservec_'+ cb_or_sb + str(i)] = df_user_vec.uservec.map(lambda x: float(x[i]))
    return df_deal_vec, df_user_vec

In [3]:
df_deal_vec_sb, df_user_vec_sb = read_embedded_vector('sb')
df_deal_vec_cb, df_user_vec_cb = read_embedded_vector('cb')

In [4]:
df_new_user_vec = pd.read_csv('df_user_sb_trainset_newmethod.csv.gz', compression='gzip')
df_new_user_vec.uservec = df_new_user_vec.uservec.map(lambda x: x[1:-1].split())
for i in range(200):
    df_new_user_vec['new_uservec_'+ 'sb' + str(i)] = df_new_user_vec.uservec.map(lambda x: float(x[i]))

In [5]:
user_vec_name = []
deal_vec_name = []
for i in range(200):
    user_vec_name.append('uservec_sb' + str(i))
    deal_vec_name.append('dealvec_sb' + str(i))

cb_user_vec_name = []
cb_deal_vec_name = []
for i in range(200):
    cb_user_vec_name.append('uservec_cb' + str(i))
    cb_deal_vec_name.append('dealvec_cb' + str(i))
new_user_vec_name = []
for i in range(200):
    new_user_vec_name.append('new_uservec_sb' + str(i))

In [6]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
data_scaler = StandardScaler()

df_deal_vec_sb[deal_vec_name] = data_scaler.fit_transform(df_deal_vec_sb[deal_vec_name])
df_user_vec_sb[user_vec_name] = data_scaler.fit_transform(df_user_vec_sb[user_vec_name])
df_deal_vec_cb[cb_deal_vec_name] = data_scaler.fit_transform(df_deal_vec_cb[cb_deal_vec_name])
df_user_vec_cb[cb_user_vec_name] = data_scaler.fit_transform(df_user_vec_cb[cb_user_vec_name])
df_new_user_vec[new_user_vec_name] = data_scaler.fit_transform(df_new_user_vec[new_user_vec_name])

In [ ]:
# random split train_test
deal_order = pd.read_csv('/home/mmde-lab/lizhi/Naive RS/' + 'df_order_add_negasmpl.gz', compression = 'gzip')
deal_order = deal_order[['account_id', 'deal_id', 'rating']]
df_order_after = pd.merge(deal_order, df_deal_vec, how='left', left_on=['deal_id'], right_on=['deal_id'])
df_order_after = pd.merge(df_order_after, df_user_vec, how='left', left_on=['account_id'], right_on=['account_id'])
df_order_after = df_order_after.dropna()
from sklearn.model_selection import train_test_split
train_order_after, test_order_after = train_test_split(df_order_after, test_size = 0.1)
train_order_after, valid_order_after = train_test_split(train_order_after, test_size = 0.2)

In [7]:
# split train_test by time
work_path = '/home/mmde-lab/lizhi/Naive RS/'
train_order_path = 'train_order_add_negasmpl.gz'
train_order_dc_path = 'train_order_add_negasmpl_dealcnt.gz'
valid_order_dc_path = 'valid_order_add_negasmpl_dealcnt.gz'
valid_order_uc_path ='valid_order_add_negasmpl.gz'
test_order_uc_path = 'test_order_add_negasmpl.gz'
test_order_dc_path = 'test_order_add_negasmpl_dealcnt.gz'
test_order_sguc_path ='test_order_simple_sample_uc.gz'
test_order_sgdc_path = 'test_order_single_sample.gz'
test_order_ful_path = 'test_order_fully_distribution.gz'

In [8]:
def merge_embedded_vec(df_path):
    df = pd.read_csv(work_path + df_path, compression='gzip')
    df = df[['account_id', 'deal_id', 'rating']]
    df_new = pd.merge(df, df_deal_vec_sb, how='left', left_on=['deal_id'], right_on=['deal_id'])
    df_new = pd.merge(df_new, df_user_vec_sb, how='left', left_on=['account_id'], right_on=['account_id'])
    df_new = pd.merge(df_new, df_deal_vec_cb, how='left', left_on=['deal_id'], right_on=['deal_id'])
    df_new = pd.merge(df_new, df_user_vec_cb, how='left', left_on=['account_id'], right_on=['account_id'])
    df_new = pd.merge(df_new, df_new_user_vec, how='left', left_on=['account_id'], right_on=['account_id'])
    df_new = df_new.dropna()
    return df_new

In [9]:
train_order = merge_embedded_vec(train_order_path)
train_order_dc = merge_embedded_vec(train_order_dc_path)
valid_order_dc = merge_embedded_vec(valid_order_dc_path)
valid_order_uc = merge_embedded_vec(valid_order_uc_path)
test_order_uc = merge_embedded_vec(test_order_uc_path)
test_order_dc = merge_embedded_vec(test_order_dc_path)
test_order_sguc = merge_embedded_vec(test_order_sguc_path)
test_order_sgdc = merge_embedded_vec(test_order_sgdc_path)

/home/mmde-lab/anaconda3/envs/whxPyEnv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [10]:
test_order_ful = merge_embedded_vec(test_order_ful_path)

/home/mmde-lab/anaconda3/envs/whxPyEnv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [11]:
feature_list = deal_vec_name + user_vec_name + cb_deal_vec_name + cb_user_vec_name

In [12]:
from sklearn.model_selection import train_test_split
train_dc, train_dev_dc = train_test_split(train_order_dc, test_size = 0.01)
train_uc, train_dev_uc = train_test_split(train_order, test_size = 0.01)

In [20]:
train_order_after = pd.merge(train_order_after, df_deal, how='left', left_on=['deal_id'], right_on=['deal_id'])
valid_order_after = pd.merge(valid_order_after, df_deal, how='left', left_on=['deal_id'], right_on=['deal_id'])
test_order_after = pd.merge(test_order_after, df_deal, how='left', left_on=['deal_id'], right_on=['deal_id'])

train_order_after = pd.merge(train_order_after, df_dealprofile, how='left', left_on=['deal_id'], right_on=['deal_id'])
valid_order_after = pd.merge(valid_order_after, df_dealprofile, how='left', left_on=['deal_id'], right_on=['deal_id'])
test_order_after = pd.merge(test_order_after, df_dealprofile, how='left', left_on=['deal_id'], right_on=['deal_id'])

deal_feature = ['deal_price', 'max_amount', 'min_amount', 'set_amount',
       'remind_mail', 'after_purchase_remind_mail',
       'max_amount_per_account', 'min_amount_per_account',
       'branch_deal_flg_0', 'branch_deal_flg_1', 'deal_type_cd_0',
       'deal_type_cd_1', 'deal_type_cd_2', 'deal_type_cd_3',
       'sales_method_0', 'sales_method_1', 'deal_category_cd_0',
       'deal_category_cd_1', 'deal_category_cd_2', 'deal_category_cd_3',
       'deal_category_cd_4', 'deal_category_cd_5', 'deal_category_cd_6',
       'deal_category_cd_7', 'deal_category_cd_8', 'deal_category_cd_9',
       'deal_category_cd_10', 'deal_category_cd_11',
       'deal_category_cd_12', 'deal_status_0', 'deal_status_1',
       'fixed_price_name_0', 'fixed_price_name_1', 'fixed_price_name_2',
       'usable_day_visible_flg_0', 'usable_day_visible_flg_1',
       'ordered_flg_0', 'ordered_flg_1', 'soldout_flg_0', 'soldout_flg_1',
       'strategy_flg_0', 'strategy_flg_1', 'account_duplicate_chk_flg_0',
       'account_duplicate_chk_flg_1', 'template_flg_0', 'template_flg_1',
       'mobile_usable_flg_0', 'mobile_usable_flg_1',
       'booking_system_flg_0', 'booking_system_flg_1', 'np_cart_flg_0',
       'np_cart_flg_1', 'sales_progress_flg_0', 'sales_progress_flg_1',
       'fixed_score_boost_flg_0', 'fixed_score_boost_flg_1',
       'wm_fixed_score_boost_flg_0', 'wm_fixed_score_boost_flg_1',
       'featured_flg_0', 'featured_flg_1', 'preferential_flg_0',
       'preferential_flg_1', 'loto_flg_0', 'loto_flg_1',
       'deal_visible_cd_0', 'deal_visible_cd_1', 'deal_visible_cd_2',
       'deal_visible_cd_3', 'deal_visible_cd_4', 'deal_visible_cd_5',
       'deal_visible_cd_6', 'sku_default_visible_flg_0',
       'sku_default_visible_flg_1', 'purchase_count_visible_flg_0',
       'purchase_count_visible_flg_1', 'api_visible_flg_0',
       'api_visible_flg_1', 'correspondence_education_flg_0',
       'correspondence_education_flg_1', 'want_visible_flg_0',
       'want_visible_flg_1', 'dummy_deal_flg_0', 'dummy_deal_flg_1',
       'max_amount_upd_flg_0', 'max_amount_upd_flg_1', 'furusato_flg_0',
       'furusato_flg_1']

dealpro_feature = ['limitation_cd_0', 'limitation_cd_1', 'limitation_cd_2',
       'limitation_cd_3', 'fixed_about_reserve_add_flg_0',
       'fixed_about_reserve_add_flg_1', 'reservation_visible_flg_0',
       'reservation_visible_flg_1', 'reservation_required_flg_0',
       'reservation_required_flg_1', 'fixed_about_gift_add_flg_0',
       'fixed_about_gift_add_flg_1', 'del_flg_0']

# ----------------------------------------------------------------------------

# NN model

In [13]:
from keras.models import Model
from keras.layers import Input, Dense, Concatenate, Flatten, BatchNormalization, Dropout
from keras.losses import binary_crossentropy, mse
from keras import regularizers
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report

In [14]:
session_user_input = Input(shape=(200,), name='sb_user_vec')
session_deal_input = Input(shape=(200,), name='sb_deal_vec')
content_user_input = Input(shape=(200,), name='cb_user_vec')
content_deal_input = Input(shape=(200,), name='cb_deal_vec')
session_concat = Concatenate()([session_user_input, session_deal_input])
content_concat = Concatenate()([content_user_input, content_deal_input])
#concat = Concatenate()([session_user_input, content_deal_input])

#fc_sb0 = Dense(512, activation='relu')(session_concat)
fc_sb1 = Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.01))(session_concat)
dropout_sb1 = Dropout(0.0)(fc_sb1)
#fc_sb2 = Dense(256, activation='relu')(fc_sb1)
fc_sb2 = Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.01))(dropout_sb1)
dropout_sb2 = Dropout(0.0)(fc_sb2)
fc_sb3 = Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.01))(dropout_sb2)
dropout_sb3 = Dropout(0.0)(fc_sb3)
fc_sb4 = Dense(64, activation='relu',kernel_regularizer=regularizers.l2(0.01))(dropout_sb3)

#fc_cb0 = Dense(512, activation='relu')(content_concat)
fc_cb1 = Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.01))(content_concat)
dropout_cb1 = Dropout(0.0)(fc_cb1)
#fc_cb2 = Dense(256, activation='relu')(fc_cb1)
fc_cb2 = Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.01))(dropout_cb1)
dropout_cb2 = Dropout(0.0)(fc_cb2)
fc_cb3 = Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.01))(dropout_cb2)
dropout_cb3 = Dropout(0.0)(fc_cb3)
fc_cb4 = Dense(64, activation='relu',kernel_regularizer=regularizers.l2(0.01))(dropout_cb3)

concat = Concatenate()([fc_sb4, fc_cb4])
dropout_con = Dropout(0.0)(concat)
output = Dense(1, activation='sigmoid',kernel_regularizer=regularizers.l2(0.01))(dropout_con)

model = Model(inputs = [ session_deal_input, session_user_input, content_deal_input, content_user_input], outputs=output)
#model = Model(inputs = [session_user_input, content_deal_input], outputs=output)
model.compile(optimizer='adam',loss=binary_crossentropy, metrics=['accuracy'])

In [15]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sb_user_vec (InputLayer)        (None, 200)          0                                            
__________________________________________________________________________________________________
sb_deal_vec (InputLayer)        (None, 200)          0                                            
__________________________________________________________________________________________________
cb_user_vec (InputLayer)        (None, 200)          0                                            
__________________________________________________________________________________________________
cb_deal_vec (InputLayer)        (None, 200)          0                                            
__________________________________________________________________________________________________
concatenat

In [34]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

check_point = ModelCheckpoint('model.hdf5', verbose=True, save_best_only = True)
early_stop = EarlyStopping(patience=3, verbose=True)

import keras.backend as K

model.fit([train_dc.loc[:, deal_vec_name].values, train_dc.loc[:, user_vec_name].values, train_dc.loc[:, cb_deal_vec_name].values, train_dc.loc[:, cb_user_vec_name].values], train_dc.rating, 
          validation_data=([valid_order_dc.loc[:, deal_vec_name].values, valid_order_dc.loc[:, user_vec_name].values, valid_order_dc.loc[:, cb_deal_vec_name].values, valid_order_dc.loc[:, cb_user_vec_name].values], valid_order_dc.rating),
          batch_size=512,
          epochs=100, 
          verbose=True,
         callbacks=[early_stop, check_point])
'''
model.fit([train_order_after.loc[:, cb_user_vec_name].values, train_order_after.loc[:, cb_deal_vec_name].values], train_order_after.rating, 
          validation_data=([valid_order_after.loc[:, cb_user_vec_name].values, valid_order_after.loc[:, cb_deal_vec_name].values], valid_order_after.rating),
          batch_size=1024,
          epochs=100, 
          verbose=True,
         callbacks=[early_stop, check_point])
'''

Train on 3134213 samples, validate on 68103 samples
Epoch 1/100
3134213/3134213 [==============================] - 143s 46us/step - loss: 0.3573 - acc: 0.9293 - val_loss: 0.9818 - val_acc: 0.6796

Epoch 00001: val_loss improved from inf to 0.98184, saving model to model.hdf5
Epoch 2/100
3134213/3134213 [==============================] - 143s 46us/step - loss: 0.3495 - acc: 0.9335 - val_loss: 0.9205 - val_acc: 0.6997

Epoch 00002: val_loss improved from 0.98184 to 0.92046, saving model to model.hdf5
Epoch 3/100
3134213/3134213 [==============================] - 143s 46us/step - loss: 0.3493 - acc: 0.9338 - val_loss: 0.8879 - val_acc: 0.7044

Epoch 00003: val_loss improved from 0.92046 to 0.88791, saving model to model.hdf5
Epoch 4/100
3134213/3134213 [==============================] - 143s 46us/step - loss: 0.3484 - acc: 0.9344 - val_loss: 1.0639 - val_acc: 0.6607

Epoch 00004: val_loss did not improve from 0.88791
Epoch 5/100
3134213/3134213 [==============================] - 143s 46us

'\nmodel.fit([train_order_after.loc[:, cb_user_vec_name].values, train_order_after.loc[:, cb_deal_vec_name].values], train_order_after.rating, \n          validation_data=([valid_order_after.loc[:, cb_user_vec_name].values, valid_order_after.loc[:, cb_deal_vec_name].values], valid_order_after.rating),\n          batch_size=1024,\n          epochs=100, \n          verbose=True,\n         callbacks=[early_stop, check_point])\n'

In [ ]:
session_user_input = Input(shape=(200,), name='sb_user_vec')
session_deal_input = Input(shape=(200,), name='sb_deal_vec')
content_user_input = Input(shape=(200,), name='cb_user_vec')
content_deal_input = Input(shape=(200,), name='cb_deal_vec')
deal_feature_input = Input(shape=(len(deal_feature)+ len(dealpro_feature),), name='deal_feature')
session_concat = Concatenate()([session_user_input, session_deal_input])
content_concat = Concatenate()([content_user_input, content_deal_input])

fc_sb0 = Dense(512, activation='relu')(session_concat)
fc_sb1 = Dense(256, activation='relu')(fc_sb0)
#dropout_1 = Dropout(0.2)(fc_1)
fc_sb2 = Dense(256, activation='relu')(fc_sb1)
fc_sb3 = Dense(128, activation='relu')(fc_sb2)
#dropout_2 = Dropout(0.2)(fc_3)
fc_sb4 = Dense(64, activation='relu')(fc_sb3)
fc_sb5 = Dense(16, activation='relu')(fc_sb4)

fc_cb0 = Dense(512, activation='relu')(content_concat)
fc_cb1 = Dense(256, activation='relu')(fc_cb0)
#dropout_1 = Dropout(0.2)(fc_1)
fc_cb2 = Dense(256, activation='relu')(fc_cb1)
fc_cb3 = Dense(128, activation='relu')(fc_cb2)
#dropout_2 = Dropout(0.2)(fc_3)
fc_cb4 = Dense(64, activation='relu')(fc_cb3)
fc_cb5 = Dense(16, activation='relu')(fc_cb4)


fc_deal0 = Dense(128, activation='relu')(deal_feature_input)
fc_deal1 = Dense(64, activation='relu')(fc_deal0)
fc_deal2 = Dense(8, activation='relu')(fc_deal1)

concat = Concatenate()([fc_sb5, fc_cb5, fc_deal2])
dense_0 = Dense(16, activation='relu')(concat)
output = Dense(1, activation='sigmoid')(dense_0)

model = Model(inputs = [session_user_input, session_deal_input, content_user_input, content_deal_input, deal_feature_input], outputs=output)
model.compile(optimizer='adam',loss=binary_crossentropy, metrics=['accuracy'])

from keras.callbacks import EarlyStopping, ModelCheckpoint

check_point = ModelCheckpoint('model.hdf5', verbose=True, save_best_only = True)
early_stop = EarlyStopping(patience=5, verbose=True)


import keras.backend as K
from keras.callbacks import LearningRateScheduler
 
def scheduler(epoch):
    # 每隔100个epoch，学习率减小为原来的1/10
    if epoch % 100 == 0 and epoch != 0:
        lr = K.get_value(model.optimizer.lr)
        K.set_value(model.optimizer.lr, lr * 0.1)
        print("lr changed to {}".format(lr * 0.1))
    return K.get_value(model.optimizer.lr)
 
reduce_lr = LearningRateScheduler(scheduler)
model.fit([train_order_after.loc[:, user_vec_name].values, train_order_after.loc[:, deal_vec_name].values, train_order_after.loc[:, cb_user_vec_name].values, train_order_after.loc[:, cb_deal_vec_name].values, train_order_after.loc[:, deal_feature+dealpro_feature].values], train_order_after.rating, 
          validation_data=([valid_order_after.loc[:, user_vec_name].values, valid_order_after.loc[:, deal_vec_name].values, valid_order_after.loc[:, cb_user_vec_name].values, valid_order_after.loc[:, cb_deal_vec_name].values, valid_order_after.loc[:, deal_feature+dealpro_feature].values], valid_order_after.rating),
          batch_size=128,
          epochs=100, 
          verbose=True,
         callbacks=[early_stop, check_point, reduce_lr])

In [35]:
model.load_weights('model.hdf5')
#cb
#pred_result = model.predict([test_order_after.loc[:, user_vec_name].values, test_order_after.loc[:, cb_deal_vec_name].values])
#sb
#pred_result = model.predict([test_order_after.loc[:, user_vec_name].values, test_order_after.loc[:, deal_vec_name].values])
#cb+sb
#pred_result = model.predict([test_order_dc.loc[:, deal_vec_name].values, test_order_dc.loc[:, new_user_vec_name].values, test_order_dc.loc[:, cb_deal_vec_name].values, test_order_dc.loc[:, cb_user_vec_name].values])

In [36]:
def model_pred(df):
    pred_result = model.predict([df.loc[:, deal_vec_name].values, df.loc[:, user_vec_name].values, df.loc[:, cb_deal_vec_name].values, df.loc[:, cb_user_vec_name].values])
    return pred_result

In [37]:
def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]
def print_result(df_name):
    pred_result = model_pred(df_name)
    for i in range(len(pred_result)):
        pred_result[i] = int(pred_result[i][0] >= 0.5)
    print(namestr(df_name, globals())[0] + '_accuracy: ',  accuracy_score(df_name.rating, pred_result))
    print(classification_report(df_name.rating, pred_result))

In [41]:
print_result(test_order_ful)

test_order_ful_accuracy:  0.7136968544008248
             precision    recall  f1-score   support

          0       1.00      0.71      0.83   3659416
          1       0.00      0.58      0.00      3352

avg / total       1.00      0.71      0.83   3662768



In [34]:
model.load_weights('model.hdf5')
pred_result = model.predict([test_order_after.loc[:, user_vec_name].values, test_order_after.loc[:, deal_vec_name].values, test_order_after.loc[:, cb_user_vec_name].values, test_order_after.loc[:, cb_deal_vec_name].values], test_order_after.loc[:, deal_feature+dealpro_feature].values)
np.save('nn_result.npy', pred_result)

In [31]:
test_order_uc.rating.value_counts()

0    3664720
1       3352
Name: rating, dtype: int64

In [42]:
pred_result = model_pred(test_order_ful)
for i in range(len(pred_result)):
    pred_result[i] = int(pred_result[i][0] >= 0.5)

In [44]:
(pred_result == 1).sum()

1049176